In [1]:
# 原始模型
import numpy as np
import torch
from torch import nn
from torchvision import datasets, transforms
from torch import optim
from torch.utils.data import DataLoader, Dataset, random_split, Subset
from tqdm import tqdm,trange
import torch.nn.functional as F

In [2]:
classes = 5  #分类
hidden_dim = 512 # rnn隐藏单元数
lr = 0.001 # 学习率
epoches = 50 #训练次数
batch_size = 16 # 每一个训练批次数量
input_dim= 54
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
class ActionDatasets(Dataset):
    def __init__(self, csv_path, transform=None, target_transform=None, pick_path = "data.npz"):
        super(ActionDatasets, self).__init__()
        self.transform = transform
        self.target_transform = target_transform
        import pandas as pd
        from glob import glob
        import os
        csvs = glob(os.path.join(csv_path, "*.csv"))
        csvs.sort()
        if len(csvs) == 0:
            raise ValueError("路径下不存在csv文件")
        df = []
        labels = []
        if os.path.exists(pick_path):
            load_data = np.load(pick_path)
            values = load_data['x']
            labels = load_data['y']
        else:
            for csv in csvs:
                label = ord(csv.split('\\')[-1][2]) - 65
                if label >= 4:
                    label -= 1
                labels.append(label)
                if type(df) == list:
                    df = pd.read_csv(csv)
                else:
                    df_tmp = pd.read_csv(csv)
                    df = pd.concat([df, df_tmp])
        
            values = df.values[:, 1:]
            labels = np.array(labels)
            np.savez(pick_path, x=values, y=labels)
            
        self.label = labels
        self.values = values
        
    def __getitem__(self, idx):
        train_data, label_data = torch.tensor(self.values[idx*140:(idx+1)*140, :],dtype=torch.float32) ,torch.tensor(self.label[idx], dtype=torch.long)

        if self.transform:
            train_data = self.transform(train_data)
        if self.target_transform:
            label_data = self.target_transform(label_data)

        return train_data ,label_data

    def __len__(self):
        return self.values.shape[0]//140


In [4]:
def create_data_loader():
    train_data_path = "./action_train/"
    # train_data_path = "D:\\temp\\augment_action_windows"
    datasets = ActionDatasets(train_data_path, transform=torch.tensor, target_transform=torch.tensor, pick_path='train.npz')
    test_datasets = ActionDatasets("./action_test/", transform=torch.tensor, target_transform=torch.tensor, pick_path='test.npz')
    split_rate = 0.8  # 训练集占整个数据集的比例
    train_len = int(split_rate * len(datasets))
    valid_len = len(datasets) - train_len

    train_sets, valid_sets = random_split(datasets, [train_len, valid_len])

    train_loader = DataLoader(train_sets, batch_size=batch_size, shuffle=True,drop_last=True,pin_memory=True)
    test_loader = DataLoader(test_datasets, batch_size=batch_size, shuffle=True,drop_last=True,pin_memory=True)
    valid_loader = DataLoader(valid_sets, batch_size=batch_size, shuffle=True,drop_last=True,pin_memory=True)

    print(f"训练集大小{len(train_sets)}， 验证集大小{len(valid_sets)}， 测试集大小{len(test_datasets)}")
    return train_loader, valid_loader, test_loader
train_loader, valid_loader, test_loader = create_data_loader()

训练集大小847， 验证集大小212， 测试集大小553


In [5]:
for data, label in train_loader:
    print(data.shape)
    print(label)
    break

torch.Size([16, 140, 54])
tensor([0, 3, 1, 0, 3, 1, 1, 4, 1, 0, 0, 2, 4, 0, 3, 3])


In [6]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, out_dim):
        super(RNN, self).__init__()
        self.pre = nn.Linear(input_dim, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, 1, batch_first=True)
        self.linear = nn.Linear(hidden_dim, out_dim)
    def forward(self, X):
        X = self.pre(X)
        out, status = self.rnn(X)
        out = self.linear(out[:, -1, :])
        return out

In [7]:
rnn = RNN(input_dim, hidden_dim, classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr=lr)

In [8]:
def GETACC(loader=valid_loader, typ_='valid'):
    rnn.eval()
    cnt = 0
    sum_valid_acc = 0
    sum_valid_loss = 0
    c = [0]*5 #action4 被识别为其他动作的数量
    if typ_ == 'test':
        errors = [0]*classes # 识别错动作的数量
    for data, label in loader:
        data = data.detach().to(device)
        label = label.detach().to(device)
        out = rnn(data)
        _, predict = torch.max(F.softmax(out), 1)
        loss = criterion(out.detach(), label)
        sum_valid_loss += loss.item()
        
        eq = (predict == label).int()
        
        if typ_ == 'test':
            for p, l in zip(predict, label):
                p = int(p.item())
                l = int(l.item())
                if p!=l : errors[l] += 1
                if l==3:
                    c[p]+=1
        acc = torch.sum(eq).item() / batch_size
        sum_valid_acc += acc
        cnt+=1
    if typ_ == 'test':
        return sum_valid_loss/cnt, sum_valid_acc/cnt, errors, c
    return sum_valid_loss/cnt, sum_valid_acc/cnt

In [9]:
for epoch in range(epoches):
    i = 0
    loss_sum = 0
    bar = tqdm(train_loader)
    for ii, (data , label) in enumerate(bar):
        rnn.train()
        data = data.to(device)
        label = label.to(device)
        out = rnn(data)
        loss = criterion(out, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        i+=1
        loss_sum += loss.item()

        if ii == len(train_loader)-1:
            valid_loss, valid_acc = GETACC(valid_loader)
            
            bar.set_description("epoch = {} train_loss = {:.3f} valid_loss = {:.3f} valid_acc = {:.3f}%".format(epoch, loss_sum/i, valid_loss,valid_acc*100))
        # test_loss,test_acc, errors = GETACC(test_loader, 'test')
        # print("test_loss = {:.3f}, test_acc = {:.3f}%, error_action = {}".format(test_loss, test_acc*100, errors))

epoch = 0 train_loss = 1.368 valid_loss = 1.361 valid_acc = 32.692%: 100%|██████████| 52/52 [00:01<00:00, 30.72it/s]
epoch = 1 train_loss = 0.954 valid_loss = 0.778 valid_acc = 59.615%: 100%|██████████| 52/52 [00:00<00:00, 67.43it/s]
epoch = 2 train_loss = 0.815 valid_loss = 0.709 valid_acc = 71.635%: 100%|██████████| 52/52 [00:00<00:00, 66.50it/s]
epoch = 3 train_loss = 0.808 valid_loss = 0.934 valid_acc = 54.808%: 100%|██████████| 52/52 [00:00<00:00, 67.41it/s]
epoch = 4 train_loss = 0.749 valid_loss = 0.580 valid_acc = 72.596%: 100%|██████████| 52/52 [00:00<00:00, 67.54it/s]
epoch = 5 train_loss = 0.593 valid_loss = 0.567 valid_acc = 73.077%: 100%|██████████| 52/52 [00:00<00:00, 66.33it/s]
epoch = 6 train_loss = 0.666 valid_loss = 0.478 valid_acc = 83.654%: 100%|██████████| 52/52 [00:00<00:00, 66.89it/s]
epoch = 7 train_loss = 0.582 valid_loss = 0.496 valid_acc = 71.154%: 100%|██████████| 52/52 [00:00<00:00, 66.93it/s]
epoch = 8 train_loss = 0.731 valid_loss = 0.468 valid_acc = 84.1

In [10]:
test_loss,test_acc, errors, c = GETACC(test_loader, 'test')
print(f"test_loss = {test_loss}, test_acc = {test_acc}")

test_loss = 0.0004096484823522213, test_acc = 1.0


In [11]:
errors

[0, 0, 0, 0, 0]